# Practical Guide to Fine-tuning Embedding Models for RAG Systems

## Load datasets

In [3]:
from datasets import load_dataset
dataset= load_dataset("dipanjanS/india_rag_wikidata", split="train")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/399 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/335k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1340 [00:00<?, ? examples/s]

In [4]:
dataset

Dataset({
    features: ['article_id', 'context', 'question', 'answer'],
    num_rows: 1340
})

In [5]:
import pandas as pd
df=pd.DataFrame(dataset)
df.head(3)

,article_id,context,question,answer
0,73985,"Basil (""Ocimum basilicum"") ( or ) is a plant o...",What is Basil commonly known as?,"Dear Friend, Basil is commonly known as Sweet ..."
1,73985,"Basil (""Ocimum basilicum"") ( or ) is a plant o...",Where is Basil originally native to?,"Dear Friend, Basil is originally native to Ind..."
2,259745,The Roerich Pact is a treaty on Protection of ...,What is the significance of the Roerich Pact?,"Dear Friend, the Roerich Pact is significant b..."


In [6]:
data_df=df[['question','context']]
data_df.head(3)

,question,context
0,What is Basil commonly known as?,"Basil (""Ocimum basilicum"") ( or ) is a plant o..."
1,Where is Basil originally native to?,"Basil (""Ocimum basilicum"") ( or ) is a plant o..."
2,What is the significance of the Roerich Pact?,The Roerich Pact is a treaty on Protection of ...


In [7]:
df_train =data_df.reset_index(drop=True)
df_eval=df_train.sample(100).reset_index(drop=True)


In [8]:
df_train.shape, df_eval.shape

((1340, 2), (100, 2))

In [9]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(df_train)
eval_dataset = Dataset.from_pandas(df_eval)

## Step 2 - Load Pre-Trained Embedding Model

In [10]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("BAAI/bge-base-en-v1.5")
model

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

## Step 2 - Choose Loss Function


In [11]:
from sentence_transformers.losses import MultipleNegativesRankingLoss
loss = MultipleNegativesRankingLoss(model)
loss

MultipleNegativesRankingLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
    (2): Normalize()
  )
  (cross_entropy_loss): CrossEntropyLoss()
)

## Step 3 - Setup Training Arguments

In [13]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
args = SentenceTransformerTrainingArguments(
    output_dir="bge-base-runs",
    max_steps=332,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=20,
    save_steps=100,
    save_total_limit=2,
    warmup_steps=1,
    learning_rate=3e-6,
    warmup_ratio=0.1,
    weight_decay=0.03,
    logging_steps=10,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,

)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Step 4 - Setup Trainer & Fine-tune Embedding Model

In [14]:
from sentence_transformers import SentenceTransformerTrainer
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [15]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sunjupskilling (sunjupskilling-sg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
20,0.082000,0.025289
40,0.049600,0.013849
60,0.024400,0.009332
80,0.048400,0.007502
100,0.022600,0.006682
120,0.019300,0.006129
140,0.014600,0.005476
160,0.054300,0.003486
180,0.025000,0.003092
200,0.012300,0.002910


TrainOutput(global_step=332, training_loss=0.032531390768891, metrics={'train_runtime': 255.9716, 'train_samples_per_second': 20.752, 'train_steps_per_second': 1.297, 'total_flos': 0.0, 'train_loss': 0.032531390768891, 'epoch': 3.9523809523809526})

In [16]:
model.save_pretrained("sunj-bge-base-en-v1.5")

In [18]:
import shutil

# Model folder ka naam jo tu save kar chuka hai
model_folder = "/content/sunj-bge-base-en-v1.5"

# Model ko zip me convert karo
shutil.make_archive("sunj-bge-base-en-v1.5", 'zip', model_folder)

print("Model successfully zipped as sunj-bge-base-en-v1.5.zip")

Model successfully zipped as sunj-bge-base-en-v1.5.zip


In [19]:
  from huggingface_hub import notebook_login

notebook_login()

In [20]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from huggingface_hub import HfApi

In [21]:
# Model directory (jisme fine-tuned model hai)
model_dir = "/content/sunj-bge-base-en-v1.5"

In [24]:
model_fine_tuned=SentenceTransformer(model_dir)

In [25]:
# Push model to Hugging Face
model_fine_tuned.push_to_hub("sunjupskilling/sunj-bge-base-en-v1.5")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/sunjupskilling/sunj-bge-base-en-v1.5/commit/dda42518ac51713d87d3af2bc295af6e6a87cec7'